# Testing statsmodels Modin Interoperability
## All the examples in this section are taken/ adapted from https://www.statsmodels.org/devel/gettingstarted.html or https://www.statsmodels.org/stable/index.html

In [1]:
import statsmodels.api as sm

import pandas as pandas

import modin.pandas as pd

from patsy import dmatrices

# import numpy as np

/Users/labanyamukhopadhyay/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/labanyamukhopadhyay/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


### Example with sm.OLS()

In [2]:
df = sm.datasets.get_rdataset("Guerry", "HistData").data
df = pd.DataFrame(df)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2023-01-03 12:32:27,095	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 


In [3]:
type(df)

modin.pandas.dataframe.DataFrame

In [4]:
vars = ['Department', 'Lottery', 'Literacy', 'Wealth', 'Region']

df = df[vars]

df[-5:]

,Department,Lottery,Literacy,Wealth,Region
81,Vienne,40,25,68,W
82,Haute-Vienne,55,13,67,C
83,Vosges,14,62,82,E
84,Yonne,51,47,30,C
85,Corse,83,49,37,NaN


In [5]:
df = df.dropna()

df[-5:]

,Department,Lottery,Literacy,Wealth,Region
80,Vendee,68,28,56,W
81,Vienne,40,25,68,W
82,Haute-Vienne,55,13,67,C
83,Vosges,14,62,82,E
84,Yonne,51,47,30,C


In [6]:
y, X = dmatrices('Lottery ~ Literacy + Wealth + Region', data=df, return_type='dataframe')

In [7]:
y = pd.DataFrame(y)
X = pd.DataFrame(X)

In [8]:
type(X)

modin.pandas.dataframe.DataFrame

In [9]:
mod = sm.OLS(y, X)    # Describe model

ValueError: unrecognized data structures: <class 'modin.pandas.dataframe.DataFrame'> / <class 'modin.pandas.dataframe.DataFrame'>

In [10]:
res = mod.fit()       # Fit model

print(res.summary())

NameError: name 'mod' is not defined

### Example with sm.ols(formula=)

In [11]:
df = pd.DataFrame({"A": [10,20,30,40,50], "B": [20, 30, 10, 40, 50], "C": [32, 234, 23, 23, 42523]})

In [12]:
# mod = sm.OLS(y, X)    # Describe model

# res = mod.fit()       # Fit model

# print(res.summary())
import statsmodels.formula.api as sm
result = sm.ols(formula="A ~ B + C", data=df).fit()
print(result.params)

Intercept    14.952480
B             0.401182
C             0.000352
dtype: float64


In [13]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     1.375
Date:                Tue, 03 Jan 2023   Prob (F-statistic):              0.421
Time:                        12:32:44   Log-Likelihood:                -18.178
No. Observations:                   5   AIC:                             42.36
Df Residuals:                       2   BIC:                             41.19
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.9525     17.764      0.842      0.4